ALS necesita que los identificadores sean valores enteros. Hay que hacer un diccionario para poder utilizar el algoritmo ALS. Usar stringindex + pipeline. 

Dado el dataset ./datos/lastfm_mini.parquet, se pide:

Imprimir por pantalla los 20 artistas que más aparecen. Agrupar por identificar de artista y hacer un sum. 

Crear un sistema de recomendación basado en el algortimo ALS visto en el notebook 04.ALS_Spark.ipynb

Calcular el usuario más a fín a el grupo ramones según nuestro modelo

Para el usuario del punto anterior conseguir sus 20 artístas más afin según nuestro modelo

Se valorará conseguir un modelo adecuado usando las técnicas vistas en clase

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkConf
sparkSession = SparkSession.builder.master("local[*]").getOrCreate()
sparkContext = sparkSession.sparkContext

In [0]:
import pandas as pd
import numpy as np
import pyspark.sql.functions as F

In [5]:
!git clone --recursive https://github.com/fjtm/Sistemas-de-recomendacion.git

Cloning into 'Sistemas-de-recomendacion'...
remote: Enumerating objects: 1601, done.
remote: Counting objects: 100% (1601/1601), done.
remote: Compressing objects: 100% (1499/1499), done.
remote: Total 1601 (delta 0), reused 1601 (delta 0), pack-reused 0
Receiving objects: 100% (1601/1601), 10.78 MiB | 16.70 MiB/s, done.


In [0]:
plays = sparkSession.read.parquet('Sistemas-de-recomendacion/datos/lastfm_mini.parquet')

In [7]:
plays.show()

+--------------------+--------------------+-----------+-----+
|             user_id|           artist_id|artist_name|plays|
+--------------------+--------------------+-----------+-----+
|8358173502fcfd30b...|382f1005-e9ab-468...|       2pac| 54.0|
|f434362a46b823536...|382f1005-e9ab-468...|       2pac|545.0|
|135750e1a2107ae20...|382f1005-e9ab-468...|       2pac| 54.0|
|41a072560f38290a2...|382f1005-e9ab-468...|       2pac|201.0|
|57da4f4db6fc10eb7...|382f1005-e9ab-468...|       2pac| 86.0|
|3f51326aec6893f15...|382f1005-e9ab-468...|       2pac| 95.0|
|ac4f934779afac87b...|382f1005-e9ab-468...|       2pac|163.0|
|858ebba231d365236...|382f1005-e9ab-468...|       2pac| 99.0|
|24efaa9b0170cda92...|382f1005-e9ab-468...|       2pac| 71.0|
|dd5b2d9f5d10d428d...|382f1005-e9ab-468...|       2pac| 18.0|
|7550b588c297f6792...|382f1005-e9ab-468...|       2pac| 47.0|
|662eee6265ed61f4a...|382f1005-e9ab-468...|       2pac|257.0|
|6e7acb2ec69628dc6...|382f1005-e9ab-468...|       2pac|388.0|
|45d4068

In [8]:
plays.groupBy(["artist_id","artist_name"]).sum().orderBy("sum(plays)", ascending = False).show(20)

+--------------------+--------------------+----------+
|           artist_id|         artist_name|sum(plays)|
+--------------------+--------------------+----------+
|b10bbbfc-cf9e-42e...|         the beatles|  878602.0|
|a74b1b7f-71a5-401...|           radiohead|  840236.0|
|83d91898-7763-47d...|          pink floyd|  492270.0|
|cc197bad-dc9c-440...|            coldplay|  489212.0|
|65f4f0c5-ef9e-490...|           metallica|  472438.0|
|9c9f1380-2516-4fc...|                muse|  449794.0|
|8bfac288-ccc5-448...|red hot chili pep...|  413867.0|
|b7ffd2af-418f-4be...|     nine inch nails|  402385.0|
|cc0b7089-c08d-4c1...|    system of a down|  394673.0|
|00a9f935-ba93-4fc...|           nightwish|  384564.0|
|f59c5520-5f46-4d2...|         linkin park|  361024.0|
|31745282-b1ea-4d6...|           in flames|  351163.0|
|0039c7ae-e1a7-4a7...| death cab for cutie|  327846.0|
|678d88b2-87b0-403...|        led zeppelin|  304345.0|
|847e8284-8582-4b0...|             placebo|  303084.0|
|72c536dc-

In [0]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
indexer_user = StringIndexer(inputCol="user_id", outputCol="user_id_index")
indexer_artist = StringIndexer(inputCol="artist_id", outputCol="artist_id_index")

In [0]:
Model = ALS(

    regParam=0.01, #mallado.iloc[0]['regParam'],
    maxIter=5, #mallado.iloc[0]['maxIter'],
    rank=10, #mallado.iloc[0]['rank'],    
    userCol="user_id_index", 
    itemCol="artist_id_index", 
    ratingCol="plays",
    coldStartStrategy="drop",
    implicitPrefs=False,
    numUserBlocks=10,
    numItemBlocks=10

)

In [0]:
pipeline = Pipeline(stages=[indexer_user, indexer_artist, Model])

In [0]:
model_fit = pipeline.fit(plays)

In [0]:
predictions = model_fit.transform(plays)

In [15]:
predictions.dtypes

[('user_id', 'string'),
 ('artist_id', 'string'),
 ('artist_name', 'string'),
 ('plays', 'double'),
 ('user_id_index', 'double'),
 ('artist_id_index', 'double'),
 ('prediction', 'float')]

In [16]:
evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="plays",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 188.9232229927965


In [0]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [0]:
als = ALS(

    userCol="user_id_index", 
    itemCol="artist_id_index", 
    ratingCol="plays",
    coldStartStrategy="drop",
    implicitPrefs=False,
    numUserBlocks=10,
    numItemBlocks=10

)

In [0]:
paramGrid = (

    ParamGridBuilder()
    .addGrid(als.rank, [10, 20, 50, 100])
    .addGrid(als.regParam, [0.01, 0.1])
    .addGrid(als.maxIter, [5, 10, 20])
    .build()

)

In [0]:
pipeline = Pipeline(stages=[indexer_user, indexer_artist])

In [0]:
plays_fit = pipeline.fit(plays)
plays_transform = plays_fit.transform(plays)

In [0]:
(training, test) = plays_transform.randomSplit([0.8, 0.2], seed=12)

In [0]:
import time

In [0]:
crossval = CrossValidator(

    estimator=als,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=3

) 

In [0]:
#t0 = time.time()
#cvModel = crossval.fit(training)
#print(time.time()-t0)

In [0]:
#results = [([{key.name: paramValue} for key, paramValue in zip(params.keys(), params.values())], metric) for params, metric in zip(cvModel.getEstimatorParamMaps(), cvModel.avgMetrics)] 

#list(pd.DataFrame([(results[i][0],results[i][1]) for i in range(len(results))]).sort_values(1).iloc[0])

In [0]:
paramGrid = (

    ParamGridBuilder()
    .addGrid(als.rank, [100, 200])
    .addGrid(als.regParam, [0.01, 0.1])
    .addGrid(als.maxIter, [10, 20])
    .build()

)

In [0]:
crossval = CrossValidator(

    estimator=als,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=3
) 

In [29]:
t0 = time.time()
cvModel1 = crossval.fit(training)
print(time.time()-t0)

Py4JJavaError: ignored

In [0]:
results = [([{key.name: paramValue} for key, paramValue in zip(params.keys(), params.values())], metric) for params, metric in zip(cvModel1.getEstimatorParamMaps(), cvModel1.avgMetrics)] 

list(pd.DataFrame([(results[i][0],results[i][1]) for i in range(len(results))]).sort_values(1).iloc[0])

In [0]:
#sparkSession.stop()